# Predicting Player Activity on a Minecraft Research Server


### Introduction

In this investigation, we will predict the total hours played by players according to the player.csv dataset. In the study led by Frank Wood, players’ actions and profiles were observed on a Minecraft server. For the purpose of this report, we will see if the data describing each player in players.csv includes 196 observations and 7 variables: (experience, subscription, “hashedEmail,” played hours, name, gender, and age)   can predict the total hours played; specifically, using the gender, age, subscription and experience.
